In [12]:
from py_ecc.optimized_bls12_381 import G1, G2, multiply, add, pairing, curve_order, FQ2, FQ12
from py_ecc.bls.hash_to_curve import hash_to_G2
from hashlib import sha256
import os

# 将 message 哈希为 G2 曲线上的点
def H1(message, dst=b'WTF-HASH-TO-CURVE-bls12-381'):
    # The DST parameter is domain separation tag for the hash function
    point_projective = hash_to_G2(message, dst, sha256)
    return point_projective

# 将 fq12 element (pairing结果) 哈希为 bytes
def H2(fq12_elem):
    # 将FQ12元素的所有系数转换为字符串并连接
    concatenated_coeffs = ''.join(str(coeff) for coeff in fq12_elem.coeffs)
    # 对连接后的字符串进行SHA256哈希处理
    hash_digest = sha256(concatenated_coeffs.encode()).digest()
    return hash_digest

# 计算两个 bytes 的 xor
def xor_bytes(bytes1, bytes2):
    """XOR two bytes objects together."""
    return bytes(a ^ b for a, b in zip(bytes1, bytes2))

# IBE 加密
def ibe_encrypt(master_pk, user_id, message):
    r = int.from_bytes(os.urandom(32), 'big') % curve_order
    
    # Compute shared secret using pairing
    P_user = H1(user_id.encode())
    shared_secret = pairing(P_user, master_pk)
    C1 = multiply(G1, r)
    C2 = xor_bytes(message, H2(shared_secret ** r))

    return C1, C2

# IBE 解密
def ibe_decrypt(user_sk, C1, C2):
    shared_secret = pairing(user_sk, C1)
    M = xor_bytes(C2, H2(shared_secret))
    return M


# 主密钥生成
master_sk = int.from_bytes(os.urandom(32), 'big') % curve_order
master_pk = multiply(G1, master_sk)

# 根据 Alice id 生成私钥
alice_id = "alice@wtf.email"
alice_sk = multiply(H1(alice_id.encode()), master_sk)

# Bob 编码消息
bob_id = "bob@wtf.email"
message = b"WTF, Alice!"
print("Original message:", message)
C1, C2 = ibe_encrypt(master_pk, alice_id, message)
print("Encryption C1:", C1)
print("Encryption C2:", C2)

# Alice 解码消息
decrypted_message = ibe_decrypt(alice_sk, C1, C2)
print("Decrypted message:", decrypted_message)

# 示例输出
# Original message: b'WTF, Alice!'
# Encryption C1: (725374226968432448365528450463540667799630567526402423184017492362194410232185198007920253993995187216521721662343, 2220163884092685170597202150534200353558680069175326450716109816259628864987790756986054551188110826100753535572404, 578944078995539478898067346383698208062908690132418954413632161636084423626880593706141384237810614307828491209731)
# Encryption C2: b'\xdd\xcfGS\xfa:j\xc2&@\x13'
# Decrypted message: b'WTF, Alice!'


Original message: b'WTF, Alice!'
Encryption C1: (725374226968432448365528450463540667799630567526402423184017492362194410232185198007920253993995187216521721662343, 2220163884092685170597202150534200353558680069175326450716109816259628864987790756986054551188110826100753535572404, 578944078995539478898067346383698208062908690132418954413632161636084423626880593706141384237810614307828491209731)
Encryption C2: b'\xdd\xcfGS\xfa:j\xc2&@\x13'
Decrypted message: b'WTF, Alice!'
